In [ ]:
import pandas as pd
import numpy as np
import warnings
from scipy.stats import chi2_contingency
warnings.filterwarnings('ignore')

In [ ]:
# Load data
train = pd.read_parquet('train.parquet')
test = pd.read_parquet('test.parquet')

In [ ]:

# Quick facts
print("DATASET OVERVIEW")
print("="*60)
print(f"Training: {train.shape}")
print(f"Test: {test.shape}")
print(f"\nTarget distribution:")
print(train['TARGET'].value_counts())
print(f"Fraud rate: {train['TARGET'].mean()*100:.2f}%")

print(f"\nFeature types:")
print(train.dtypes.value_counts())

print(f"\nMissing values:")
missing = train.isnull().sum().sum()
print(f"Total missing cells: {missing:,}")
print(f"Features with missing: {(train.isnull().sum() > 0).sum()}")

# Save this info - you'll need it tomorrow
print("\n📝 Write down:")
print("- Fraud rate:", train['TARGET'].mean()*100)
print("- Features with missing data:", (train.isnull().sum() > 0).sum())

Verificar quais as variaveis com alto valor de nullos

In [ ]:
feat_nulls = train.isnull().sum()
missing_pct = (feat_nulls / len(train) * 100).sort_values(ascending=False)
missing_pct = missing_pct[missing_pct > 0]
high_missing = missing_pct[missing_pct > 40]
moderate_missing = missing_pct[(missing_pct > 5) & (missing_pct <= 40)]
low_missing = missing_pct[missing_pct <= 5]

print(f"\n📊 RESUMO:")
print(f"Total features com missing: {len(missing_pct)}")
print(f"  🔴 Alto (>40%):      {len(high_missing)} features → DROPAR")
print(f"  🟡 Moderado (5-40%): {len(moderate_missing)} features → FLAG + IMPUTAR")
print(f"  🟢 Baixo (<5%):      {len(low_missing)} features → IMPUTAR")


print(f"\n{'='*60}")
print("📋 LISTAS PARA CLEANING:")
print("="*60)

print(f"\n# Features para dropar")
print(f"features_to_drop = {high_missing.index.tolist()}")

print(f"\n# Features para criar flag + imputar")  
print(f"features_to_flag = {moderate_missing.index.tolist()}")

print(f"\n# Features para imputar simples")
print(f"features_to_impute = {low_missing.index.tolist()}")

Verificar se as high_missing numericas tem alguma correlacao relevante com o target.

In [ ]:
numerical_high_missing = [f for f in high_missing.index if train[f].dtype in ['int64', 'float64']]

correlations = train[numerical_high_missing + ['TARGET']].corr()['TARGET'].drop('TARGET')
correlations = correlations.abs().sort_values(ascending=False)
for feat, corr in correlations.items():
    if corr > 0.3:
        print(f"Feature: {feat:20} | Correlation with TARGET: {corr:.4f}")

Variaveis numericas high_missing 

In [ ]:
significant_features = []
less_significant_features = []
not_significant_features = []
for feat in numerical_high_missing:
    missing = missing_pct[feat]
    has_data = train[feat].notna()
    # Contingency table:
    #                 Fraud=0    Fraud=1
    # Has data          a          b
    # No data           c          d
        
    contingency = pd.crosstab(has_data, train['TARGET'])
        
    # Chi-square test
    chi2, p_value, dof, expected = chi2_contingency(contingency)
        
     # Calculate fraud rates
    fraud_rate_has = train[has_data]['TARGET'].mean() * 100
    fraud_rate_no = train[~has_data]['TARGET'].mean() * 100
    diff = abs(fraud_rate_has - fraud_rate_no)        
    # Effect size (Cramér's V)  
    n = contingency.sum().sum()
    cramers_v = np.sqrt(chi2 / n)
        
    print(f"📊 {feat}:")
    print(f"   Missing: {missing:.1f}%")
    print(f"   Fraud quando TEM dados:     {fraud_rate_has:6.2f}%")
    print(f"   Fraud quando NÃO TEM dados: {fraud_rate_no:6.2f}%")
    print(f"   Diferença:                  {diff:6.2f}pp")
    print(f"   Chi-square p-value:         {p_value:.6f}")
    print(f"   Cramér's V (effect size):   {cramers_v:.4f}")
        
    # Decision criteria:
    # p-value < 0.05 (estatisticamente significativo)
    # E diferença > 1pp (praticamente relevante)
        
    if p_value < 0.05 and diff > 1.0:
        print(f"   ✅ SIGNIFICATIVO! Missingness É preditiva!")
        print(f"   → MANTER + criar flag '_is_missing'")
        significant_features.append(feat)
    elif p_value < 0.05:
        print(f"   ⚠️  Estatisticamente significativo mas efeito pequeno")
        print(f"   → Considerar dropar (efeito muito fraco)")
        less_significant_features.append(feat)
    else:
        print(f"   ❌ NÃO significativo (pode ser acaso)")
        print(f"   → DROPAR")
        not_significant_features.append(feat)

Drop das features sem significancia 

In [ ]:
for feat in not_significant_features:
    print(f"Dropping feature: {feat} due to lack of significance in missingness analysis.")
    train.drop(feat, axis=1)    

Verificar as features menos significantes antes de rejeitar

In [54]:
for feat in less_significant_features:
    print(feat)

credit_ratio
total_amt_drawings
total_drawings
total_paid_instalment
avg_card_limit
min_card_limit
max_card_limit
avg_amt_drawings
max_amt_drawings
cnt_months_neutral
cnt_signed_months
cnt_active_months
cnt_pd_months
total_dpd
max_dpd
cnt_refused_months
cnt_demand_months
amt_interest_receivable
max_card_balance
avg_card_balance
min_card_balance
avg_receivable
amt_times_underpaid
total_paid


Devido a natureza da features retirar todas nao e boa ideia, aplicarei um filtro que selecionara  todas aquelas com valores nulos acima de 70% para sere rejeitadas

In [57]:
for feat in less_significant_features:
    missing = missing_pct[feat]
    if missing > 70:
        print(feat)

credit_ratio
total_amt_drawings
total_drawings
total_paid_instalment
avg_card_limit
min_card_limit
max_card_limit
avg_amt_drawings
max_amt_drawings
cnt_months_neutral
cnt_signed_months
cnt_active_months
cnt_pd_months
total_dpd
max_dpd
cnt_refused_months
cnt_demand_months
amt_interest_receivable
max_card_balance
avg_card_balance
min_card_balance
avg_receivable
amt_times_underpaid
total_paid


Ao que parece todas elas tem missing ratio maior que 70% nesse caso a decisao ideal sera rejeitar estas variaveis.